In [143]:
from src.smth2matrix.shift2zero import shift2zero
import numpy as np
from math import ceil, floor

def line(p1, p2):
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0]*p2[1] - p2[0]*p1[1])
    return A, B, -C

def intersection(points1, points2):
    L1 = line(points1[0], points1[1])
    L2 = line(points2[0], points2[1])
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return x,y
    else:
        return False

In [144]:
h = 1
points = np.array([[0.3, 3], [1, 0], [2.1, 0], [3, 3.7]])

size_of_sides = shift2zero(points)
n_x = ceil(size_of_sides[0] / h)
n_y = ceil(size_of_sides[1] / h)

lines = []
for _ in range(n_y+1):
    lines.append([])

for k in range(0, n_y+1):
    for i0 in range(points.shape[0]):
        if i0 == points.shape[0]-1:
            i1 = 0
        else:
            i1 = (i0 + 1)
        #вершина лежит на линии
        if points[i0][1] == k*h:
            lines[k].append(points[i0][0])
        #ребро пересекает линию
        if points[i0][1] < k*h < points[i1][1] or points[i1][1] < k*h < points[i0][1]:
            print(i0, k)
            lines[k].append(intersection([points[i0], points[i1]], [[0, k*h], [n_x, k*h]])[0])
            print(intersection([points[i0], points[i1]], [[0, k*h], [n_x, k*h]])[0])



0 1
0.46666666666666656
2 1
2.043243243243243
0 2
0.23333333333333328
2 2
2.2864864864864862
2 3
2.5297297297297296


In [145]:
lines

[[0.7, 1.8],
 [0.46666666666666656, 2.043243243243243],
 [0.23333333333333328, 2.2864864864864862],
 [0.0, 2.5297297297297296],
 []]